In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

## Step 1: Download BigQuery data to our notebook

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.936641,False,30,1,38.0
1,8.624484,True,34,1,39.0
2,6.937947,True,23,1,39.0
3,6.935743,True,18,1,39.0
4,8.318041,False,23,1,40.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9995.000000,10000.0000,10000.000000,9885.000000
mean,7.263349,27.1810,1.035300,38.705311
std,1.315565,6.1537,0.196616,2.567750
min,0.507063,12.0000,1.000000,18.000000
25%,6.624891,22.0000,1.000000,38.000000
50%,7.374463,27.0000,1.000000,39.000000
75%,8.062305,32.0000,1.000000,40.000000
max,12.625874,48.0000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5198
False    4802
Name: is_male, dtype: int64

## Step 2: Prepare the dataset for training

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,0,33,1,38.0
6130,0,20,1,39.0
5986,1,37,1,33.0
7681,1,28,1,39.0
4913,0,27,1,39.0


# 5. Build and train a TensorFlow model

## Step 1: Split your data into train and test sets

In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

## Step 2: Build and train the TensorFlow model

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6669 samples, validate on 742 samples
Epoch 1/10
6669/6669 [==============================] - 2s 298us/sample - loss: 2.0583 - mae: 1.0791 - mse: 2.0583 - val_loss: 4.6041 - val_mae: 1.8978 - val_mse: 4.6041
Epoch 2/10
6669/6669 [==============================] - 1s 119us/sample - loss: 1.7033 - mae: 1.0230 - mse: 1.7033 - val_loss: 2.1506 - val_mae: 1.1992 - val_mse: 2.1506
Epoch 3/10
6669/6669 [==============================] - 1s 118us/sample - loss: 1.5925 - mae: 0.9908 - mse: 1.5925 - val_loss: 1.2286 - val_mae: 0.8594 - val_mse: 1.2286
Epoch 4/10
6669/6669 [==============================] - 1s 122us/sample - loss: 1.5155 - mae: 0.9669 - mse: 1.5155 - val_loss: 2.3943 - val_mae: 1.2770 - val_mse: 2.3943
Epoch 5/10
6669/6669 [==============================] - 1s 120us/sample - loss: 1.4990 - mae: 0.9616 - mse: 1.4990 - val_loss: 1.2717 - val_mae: 0.8777 - val_mse: 1.2717
Epoch 6/10
6669/6669 [==============================] - 1s 117us/sample - loss: 1.4305 - mae: 0.9391 - 

## Step 3: Generate predictions on test examples

In [12]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.517496
Actual val:  6.77480531126

Predicted val:  6.332738
Actual val:  8.50102482272

Predicted val:  6.478193
Actual val:  7.62578964258

Predicted val:  6.3355665
Actual val:  8.919903120519999

Predicted val:  6.1043735
Actual val:  6.75496370768

Predicted val:  6.849037
Actual val:  7.25100379718

Predicted val:  7.0386133
Actual val:  8.68841774542

Predicted val:  6.813413
Actual val:  8.24969784404

Predicted val:  5.733832
Actual val:  8.24969784404

Predicted val:  6.608897
Actual val:  9.77529669708



## Step 4: Use git diff and commit your changes

In [ ]:
# use terminal to add the modified version to githu